# Results comparison between Trajectron++ and our method
Comparing the results of our method with the Trajectron++ paper's method. The goal of the comparison is to evaluate Trajectron++ on a different dataset than it was trained on to allow more fair comparison.

In [2]:
import sys
import os
import pandas as pd
import numpy as np
import generative_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv

## Helper functions

In [3]:
def calculate_FDE(pred_x, pred_y, test_x, test_y):

    final_displacement_x = pred_x[-1] - test_x[-1]
    final_displacement_y = pred_y[-1] - test_y[-1]
    FDE = np.sqrt(final_displacement_x**2 + final_displacement_y**2)
    
    return FDE

def calculate_ADE(pred_x, pred_y, test_x, test_y):
    total_displacement_error = 0
    for point_idx in range(len(test_x)):
        displacement_error = np.sqrt((pred_x[point_idx] - test_x[point_idx])**2 + (pred_y[point_idx] - test_y[point_idx])**2)
        total_displacement_error += displacement_error

    return total_displacement_error/len(pred_x)

## The evaluation logic for Trajectron++ loops over the frames and predicts the future trajectories 
## for each node present in the current frame
## Each node has to have at least 7 historical points and 12 future points
def get_total_predictable_slices(data):
    total_predictable_steps = 0
    for i in pd.unique(data.node_id):
        #print(len(test[test.node_id == i]))
        total_predictable_steps += len(data[data.node_id == i]) - 19
    return total_predictable_steps

In [4]:
def process_data(input_data):
    data = input_data.copy()
    data['frame_id'] = pd.to_numeric(data['frame_id'], downcast='integer')
    data['track_id'] = pd.to_numeric(data['track_id'], downcast='integer')

    data['frame_id'] = data['frame_id'] // 10

    data['frame_id'] -= data['frame_id'].min()

    data['node_type'] = 'PEDESTRIAN'
    data['node_id'] = data['track_id'].astype(str)
    data.sort_values('frame_id', inplace=True)

    data['pos_x'] = data['pos_x'] - data['pos_x'].mean()
    data['pos_y'] = data['pos_y'] - data['pos_y'].mean()
    
    # Select only such nodes which have enough data to predict on (8 historical timesteps, 12 future)
    v = data.node_id.value_counts()
    data = data[data.node_id.isin(v.index[v.gt(19)])]
    
    return data

## Method evaluation logic

In [5]:
def evaluate_our_method(data, params, dataset_title=''):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='Ours - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y, _ = generative_model.predict(x_data, y_data, params, trajectory_length=12)
                    
                    # This section is for producing a single trajectory (averaging all representative preds)
                    avg_x = np.mean(all_pred_x, axis=0)
                    avg_y = np.mean(all_pred_y, axis=0)
                    avg_fde = calculate_FDE(avg_x, avg_y, x_gt, y_gt)
                    avg_ade = calculate_ADE(avg_x, avg_y, x_gt, y_gt)
                        
                    best_fde = None
                    best_ade = None

                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of, avg_fde, avg_ade

In [6]:
def evaluate_cvm_with_scenarios(data, dataset_title='', history_length=8):
    tot = 0
    our_fde_best_of = []
    our_ade_best_of = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    all_pred_x, all_pred_y = [], []
                    
                    # CVM
                    for i in range(20):
                        history_x = x_data[-history_length:]
                        history_y = y_data[-history_length:]
                        assert len(history_x) == history_length
                        
                        if i == 0:
                            vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                            vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]
                        else:
                            vel_x = [(history_x[i] - history_x[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_x))]
                            vel_y = [(history_y[i] - history_y[i-1]) + np.random.normal(0, 1) for i in range(1, len(history_y))]
                        
                        assert len(vel_x) == history_length-1
                        avg_vel_x = np.mean(vel_x)
                        avg_vel_y = np.mean(vel_y)
                        
                        pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                        pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                        assert len(pred_x) == 12
                        
                        all_pred_x.append(pred_x)
                        all_pred_y.append(pred_y)
                    

                    best_fde = None
                    best_ade = None
                    for i in range(len(all_pred_x)):
                        current_pred_x = all_pred_x[i]
                        current_pred_y = all_pred_y[i]

                        fde = calculate_FDE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_fde == None or fde < best_fde:
                            best_fde = fde

                        ade = calculate_ADE(current_pred_x, current_pred_y, x_gt, y_gt)
                        if best_ade == None or ade < best_ade:
                            best_ade = ade

                    our_fde_best_of.append(best_fde)
                    our_ade_best_of.append(best_ade)
                    
    return our_fde_best_of, our_ade_best_of

def evaluate_cvm(data, dataset_title='', history_length=8):
    tot = 0
    all_fde = []
    all_ade = []

    for frame_id in tqdm(pd.unique(data.frame_id), desc='CVM - ' + dataset_title):

        frame_data = data[data.frame_id == frame_id]
        #print(frame_data)
        for node_id in pd.unique(frame_data.node_id):
            # Check if at least 7 historical points are present
            # PS: It might be so that the prediction starts at the 8th step instead of 7th? Edited the code to do this at the moment
            if len(data[((data.node_id == node_id) & (data.frame_id <= frame_id))]) >= 8:
                # Not sure why there has to be more than 12 frames to the future (at least 13) but it's the
                # only way to get the number of trajectron++ eval predictions to match up
                if len(data[((data.node_id == node_id) & (data.frame_id > frame_id))]) >= 12:
                    tot += 1
                    node_history_data = data[((data.node_id == node_id) & (data.frame_id <= frame_id) & (data.frame_id >= frame_id-7))]
                    node_gt_data = data[((data.node_id == node_id) & (data.frame_id > frame_id) & (data.frame_id <= frame_id+12))]

                    x_data = list(node_history_data.pos_x)
                    y_data = list(node_history_data.pos_y)
                    assert len(x_data) == 8

                    x_gt = list(node_gt_data.pos_x)
                    y_gt = list(node_gt_data.pos_y)
                    assert len(x_gt) == 12

                    history_x = x_data[-history_length:]
                    history_y = y_data[-history_length:]
                    assert len(history_x) == history_length

                    vel_x = [history_x[i] - history_x[i-1] for i in range(1, len(history_x))]
                    vel_y = [history_y[i] - history_y[i-1] for i in range(1, len(history_y))]

                    assert len(vel_x) == history_length-1
                    avg_vel_x = np.mean(vel_x)
                    avg_vel_y = np.mean(vel_y)

                    pred_x = [x_data[-1] + i*avg_vel_x for i in range(1, 13)]
                    pred_y = [y_data[-1] + i*avg_vel_y for i in range(1, 13)]
                    assert len(pred_x) == 12

                    fde = calculate_FDE(pred_x, pred_y, x_gt, y_gt)
                    ade = calculate_ADE(pred_x, pred_y, x_gt, y_gt)

                    all_fde.append(fde)
                    all_ade.append(ade)
                    
    return all_fde, all_ade


## Automated results comparison

In [7]:
def read_trajectron_data(trajectron_resultset_name, base_folder='./trajectron++/results_paper_version/', suffix='best_of'):
    trajectron_fde = []
    with open(base_folder + trajectron_resultset_name + '_fde_' + suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_fde.append(float(row['value']))

    trajectron_ade = []
    with open(base_folder + trajectron_resultset_name + '_ade_'+ suffix + '.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            trajectron_ade.append(float(row['value']))
            
    return trajectron_fde, trajectron_ade

In [10]:
def evaluate_all_datasets(our_method_params, files, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=False, evaluate_with_interactions=False):
    base_path = './raw_data/'

    ours_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    trajectron_ar3_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_long_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}
    cvm_short_results = {'BEST_OF_20': {'FDE': [], 'ADE': []}, 'MOST_LIKELY': {'FDE': [], 'ADE': []}}

    for file_idx, file in enumerate(files):
        data = pd.read_csv(base_path + file, sep='\t', index_col=False, header=None)
        data.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

        data = process_data(data)

        ## Trajectron
        trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[file_idx])
        trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
        
        if evaluate_most_likely:
            trajectron_fde, trajectron_ade = read_trajectron_data(trajectron_resultset_names[file_idx], suffix='most_likely')
            trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
            trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))
        
        ## Trajectron with interactions
        if evaluate_with_interactions:
            trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_ar3_resultset_names[file_idx])
            trajectron_ar3_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_ar3_fde))
            trajectron_ar3_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ar3_ade))

            if evaluate_most_likely:
                trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data(trajectron_resultset_names[file_idx], suffix='most_likely')
                trajectron_ar3_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_ar3_fde))
                trajectron_ar3_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ar3_ade))

        # make sure that there is no discrepancy between our data processing and trajectron evaluation results size
        num_predictable_trajectories = get_total_predictable_slices(data)
        assert len(trajectron_fde) == num_predictable_trajectories
        assert len(trajectron_ade) == num_predictable_trajectories

        ## Ours
        our_fde_best_of_20, our_ade_best_of_20, our_fde_single, our_ade_single = evaluate_our_method(data, our_method_params[file_idx], dataset_title=trajectron_resultset_names[file_idx])
        ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
        ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))
        
        if evaluate_most_likely:
            ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_single))
            ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_single))

        ## CVM
        cvm_fde_best_of, cvm_ade_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[file_idx])
        cvm_fde_short_history_best_of, cvm_ade_short_history_best_of = evaluate_cvm_with_scenarios(data, dataset_title=trajectron_resultset_names[file_idx], history_length=2)
        cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
        cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
        cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
        cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))
        
        if evaluate_most_likely:
            cvm_fde, cvm_ade = evaluate_cvm(data, dataset_title=trajectron_resultset_names[file_idx])
            cvm_fde_short_history, cvm_ade_short_history = evaluate_cvm(data, dataset_title=trajectron_resultset_names[file_idx], history_length=2)
            cvm_long_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde))
            cvm_long_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade))
            cvm_short_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_short_history))
            cvm_short_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_short_history))
    
    ## Univ is a separate case as it has 2 scenes
    filename1 = './raw_data/univ/test/students001.txt'
    filename2 = './raw_data/univ/test/students003.txt'
    univ_data_1 = pd.read_csv(filename1, sep='\t', index_col=False, header=None)
    univ_data_2 = pd.read_csv(filename2, sep='\t', index_col=False, header=None)
    univ_data_1.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']
    univ_data_2.columns = ['frame_id', 'track_id', 'pos_x', 'pos_y']

    univ_data_1 = process_data(univ_data_1)
    univ_data_2 = process_data(univ_data_2)

    ## Ours
    our_fde_best_of_1, our_ade_best_of_1, our_fde_ml_1, our_ade_ml_1 = evaluate_our_method(univ_data_1, our_method_params[-1], dataset_title='univ 1')
    our_fde_best_of_2, our_ade_best_of_2, our_fde_ml_2, our_ade_ml_2 = evaluate_our_method(univ_data_2, our_method_params[-1], dataset_title='univ 2')

    # This is probably kind of wrong? calculating the fde of either scene separately and averaging them is too simple? should at least weighted average?
    our_fde_best_of_20 = our_fde_best_of_1 + our_fde_best_of_2
    our_ade_best_of_20 = our_ade_best_of_1 + our_ade_best_of_2

    ours_results['BEST_OF_20']['FDE'].append(np.mean(our_fde_best_of_20))
    ours_results['BEST_OF_20']['ADE'].append(np.mean(our_ade_best_of_20))
    
    if evaluate_most_likely:
        our_fde_ml = our_fde_ml_1 + our_fde_ml_2
        our_ade_ml = our_ade_ml_1 + our_ade_ml_2

        ours_results['MOST_LIKELY']['FDE'].append(np.mean(our_fde_ml))
        ours_results['MOST_LIKELY']['ADE'].append(np.mean(our_ade_ml))

    ## CVM
    
    cvm_fde_best_of_1, cvm_ade_best_of_1 = evaluate_cvm_with_scenarios(univ_data_1, dataset_title='univ 1')
    cvm_fde_best_of_2, cvm_ade_best_of_2 = evaluate_cvm_with_scenarios(univ_data_2, dataset_title='univ 2')

    cvm_fde_best_of = cvm_fde_best_of_1 + cvm_fde_best_of_2
    cvm_ade_best_of = cvm_ade_best_of_1 + cvm_ade_best_of_2

    cvm_fde_short_hist_best_of_1, cvm_ade_short_hist_best_of_1 = evaluate_cvm_with_scenarios(univ_data_1, dataset_title='univ 1', history_length=2)
    cvm_fde_short_hist_best_of_2, cvm_ade_short_hist_best_of_2 = evaluate_cvm_with_scenarios(univ_data_2, dataset_title='univ 2', history_length=2)

    cvm_fde_short_history_best_of = cvm_fde_short_hist_best_of_1 + cvm_fde_short_hist_best_of_2
    cvm_ade_short_history_best_of = cvm_ade_short_hist_best_of_1 + cvm_ade_short_hist_best_of_2

    cvm_long_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_best_of))
    cvm_long_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_best_of))
    cvm_short_results['BEST_OF_20']['FDE'].append(np.mean(cvm_fde_short_history_best_of))
    cvm_short_results['BEST_OF_20']['ADE'].append(np.mean(cvm_ade_short_history_best_of))
    
    if evaluate_most_likely:
        cvm_fde_ml_1, cvm_ade_ml_1 = evaluate_cvm(univ_data_1, dataset_title='univ 1')
        cvm_fde_ml_2, cvm_ade_ml_2 = evaluate_cvm(univ_data_2, dataset_title='univ 2')

        cvm_fde_ml = cvm_fde_ml_1 + cvm_fde_ml_2
        cvm_ade_ml = cvm_ade_ml_1 + cvm_ade_ml_2

        cvm_fde_short_hist_ml_1, cvm_ade_short_hist_ml_1 = evaluate_cvm(univ_data_1, dataset_title='univ 1', history_length=2)
        cvm_fde_short_hist_ml_2, cvm_ade_short_hist_ml_2 = evaluate_cvm(univ_data_2, dataset_title='univ 2', history_length=2)

        cvm_fde_short_history_ml = cvm_fde_short_hist_ml_1 + cvm_ade_short_hist_ml_1
        cvm_ade_short_history_ml = cvm_ade_short_hist_ml_1 + cvm_ade_short_hist_ml_2

        cvm_long_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_ml))
        cvm_long_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_ml))
        cvm_short_results['MOST_LIKELY']['FDE'].append(np.mean(cvm_fde_short_history_ml))
        cvm_short_results['MOST_LIKELY']['ADE'].append(np.mean(cvm_ade_short_history_ml))  
        
    ## Trajectron
    trajectron_fde, trajectron_ade = read_trajectron_data('univ_vel')
    trajectron_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_fde))
    trajectron_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ade))
    
    if evaluate_most_likely:
        trajectron_fde, trajectron_ade = read_trajectron_data('univ_vel', suffix='most_likely')
        trajectron_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_fde))
        trajectron_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ade))

    if evaluate_with_interactions:
        ## Trajectron with interactions
        trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data('univ_ar3')
        trajectron_ar3_results['BEST_OF_20']['FDE'].append(np.mean(trajectron_ar3_fde))
        trajectron_ar3_results['BEST_OF_20']['ADE'].append(np.mean(trajectron_ar3_ade))

        if evaluate_most_likely:
            trajectron_ar3_fde, trajectron_ar3_ade = read_trajectron_data('univ_ar3', suffix='most_likely')
            trajectron_ar3_results['MOST_LIKELY']['FDE'].append(np.mean(trajectron_ar3_fde))
            trajectron_ar3_results['MOST_LIKELY']['ADE'].append(np.mean(trajectron_ar3_ade))
    
    return [
        ours_results,
        trajectron_results,
        trajectron_ar3_results,
        cvm_long_results,
        cvm_short_results
    ]

### Running the evaluation

In [11]:
eth_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.2, 
    'ANGLE_CHANGE_PROB': 0.25, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}

hotel_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 0.5, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}

zara1_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.25, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}

zara2_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.1,
    'VELOCITY_CHANGE_NOISE': 0.05, 
    'ANGLE_CHANGE_PROB': 0.25, 
    'ANGLE_CHANGE_NOISE': 2, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}

univ_params = {
    'NOISE': 0.05, 
    'NO_OF_TRAJECTORIES': 300, 
    'CONST_VEL_MODEL_PROB': 0.5, 
    'STOP_PROB': 0.05, 
    'DISCOUNT_AVG_PROB': 1.0, 
    'DISCOUNT_LOWER_BOUND': 0.2, 
    'VELOCITY_CHANGE_PROB': 0.2,
    'VELOCITY_CHANGE_NOISE': 0.1, 
    'ANGLE_CHANGE_PROB': 0.2, 
    'ANGLE_CHANGE_NOISE': 2.5, 
    'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
    'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
}


our_method_params = [eth_params, hotel_params, zara1_params, zara2_params, univ_params]

files = [
    'eth/test/biwi_eth.txt', 
    'hotel/test/biwi_hotel.txt', 
    'zara1/test/crowds_zara01.txt', 
    'zara2/test/crowds_zara02.txt'
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel', 
    #'zara1_vel', 
    #'zara2_vel'
    'zara1_vel_modified', 
    'zara2_vel_modified'
]

trajectron_ar3_resultset_names = [
    'eth_ar3', 
    'hotel_ar3', 
    'zara1_ar3', 
    'zara2_ar3'
]

res = evaluate_all_datasets(our_method_params, files, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)

CVM - univ 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [00:39<00:00, 13.61it/s]


In [12]:
ours_results = res[0]
trajectron_results = res[1]
trajectron_ar3_results = res[2]
cvm_long_results = res[3]
cvm_short_results = res[4]

In [13]:
sgan_results = {
    'BEST_OF_20': {
        'FDE': [1.52, 1.61, 0.69, 0.84, 1.26], 
        'ADE': [0.81, 0.72, 0.34, 0.42, 0.60]
    }, 
    'MOST_LIKELY': {
        'FDE': [2.21, 2.18, 0.91, 1.11, 1.28], 
        'ADE': [ 1.13, 1.01, 0.42, 0.52, 0.60]
    }
}

In [14]:
index = [
    'ETH', 
    'Hotel', 
    'Zara 1', 
    'Zara 2',
    'Univ'
]

df_data_best_of_20_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
    'Social-GAN': pd.Series(sgan_results['BEST_OF_20']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['FDE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
}

df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)

df_data_best_of_20_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
    'Social-GAN': pd.Series(sgan_results['BEST_OF_20']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['ADE'], index = index),
    'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
}

df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)

df_data_most_likely_fde = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
    'Social-GAN': pd.Series(sgan_results['MOST_LIKELY']['FDE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['FDE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
}

df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)

df_data_most_likely_ade = {
    'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
    'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
    'Social-GAN': pd.Series(sgan_results['MOST_LIKELY']['ADE'], index = index),
    'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
    #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['ADE'], index = index),
    'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
}

df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)

### Best of 20 - FDE

In [15]:
df_best_of_20_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,1.128421,1.758176,1.520000,0.812384,0.652284
Hotel,0.370271,0.588532,1.610000,0.197178,0.191122
Zara 1,0.803341,0.871944,0.690000,0.341834,0.453783
Zara 2,0.555668,0.653450,0.840000,0.253352,0.334395
Univ,0.879122,1.057535,1.260000,0.450190,0.445308


### Best of 20 - ADE

In [16]:
df_best_of_20_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,0.642131,0.902863,0.810000,0.396031,0.439143
Hotel,0.206589,0.310019,0.720000,0.115423,0.131611
Zara 1,0.422718,0.402043,0.340000,0.154697,0.258637
Zara 2,0.293995,0.304825,0.420000,0.115362,0.195101
Univ,0.470956,0.491874,0.600000,0.204697,0.269096


### Single output - FDE

In [17]:
df_most_likely_fde.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,2.303287,2.281890,2.210000,1.615308,1.568917
Hotel,0.462310,0.614198,2.180000,0.498741,1.187672
Zara 1,1.131947,0.952377,0.910000,0.769595,1.117246
Zara 2,0.859903,0.724414,1.110000,0.589387,1.917394
Univ,1.370131,0.740114,1.280000,1.204670,5.003500


### Single output - ADE

In [18]:
df_most_likely_ade.style.highlight_min(color = 'lightgreen', axis = 1)

,CVM (8pt history),CVM (2pt history),Social-GAN,Trajectron++,Ours
ETH,1.101935,1.075458,1.130000,0.694815,0.714896
Hotel,0.243328,0.319356,1.010000,0.224216,0.611722
Zara 1,0.551512,0.427223,0.420000,0.297096,0.216501
Zara 2,0.421038,0.323937,0.520000,0.227388,0.721829
Univ,0.676115,0.524190,0.600000,0.467850,2.612051


## Experimenting with different amount of generated trajectories

In [138]:
files = [
    'eth/test/biwi_eth.txt', 
    'hotel/test/biwi_hotel.txt', 
    'zara1/test/crowds_zara01.txt', 
    'zara2/test/crowds_zara02.txt'
]

trajectron_resultset_names = [
    'eth_vel', 
    'hotel_vel', 
    'zara1_vel', 
    'zara2_vel'
]

trajectron_ar3_resultset_names = [
    'eth_ar3', 
    'hotel_ar3', 
    'zara1_ar3', 
    'zara2_ar3'
]

import time


all_results = []
for no_of_trajectories in [100, 300, 1000, 2000, 5000]:
    eth_params = {
        'NOISE': 0.05, 
        'NO_OF_TRAJECTORIES': no_of_trajectories, 
        'CONST_VEL_MODEL_PROB': 0.5, 
        'STOP_PROB': 0.05, 
        'DISCOUNT_AVG_PROB': 1.0, 
        'DISCOUNT_LOWER_BOUND': 0.2, 
        'VELOCITY_CHANGE_PROB': 0.2,
        'VELOCITY_CHANGE_NOISE': 0.2, 
        'ANGLE_CHANGE_PROB': 0.25, 
        'ANGLE_CHANGE_NOISE': 2, 
        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
    }

    hotel_params = {
        'NOISE': 0.05, 
        'NO_OF_TRAJECTORIES': no_of_trajectories, 
        'CONST_VEL_MODEL_PROB': 0.5, 
        'STOP_PROB': 0.05, 
        'DISCOUNT_AVG_PROB': 0.5, 
        'DISCOUNT_LOWER_BOUND': 0.2, 
        'VELOCITY_CHANGE_PROB': 0.2,
        'VELOCITY_CHANGE_NOISE': 0.1, 
        'ANGLE_CHANGE_PROB': 0.2, 
        'ANGLE_CHANGE_NOISE': 2, 
        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
    }

    zara1_params = {
        'NOISE': 0.05, 
        'NO_OF_TRAJECTORIES': no_of_trajectories, 
        'CONST_VEL_MODEL_PROB': 0.5, 
        'STOP_PROB': 0.05, 
        'DISCOUNT_AVG_PROB': 1.0, 
        'DISCOUNT_LOWER_BOUND': 0.2, 
        'VELOCITY_CHANGE_PROB': 0.2,
        'VELOCITY_CHANGE_NOISE': 0.1, 
        'ANGLE_CHANGE_PROB': 0.25, 
        'ANGLE_CHANGE_NOISE': 2.5, 
        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
    }

    zara2_params = {
        'NOISE': 0.05, 
        'NO_OF_TRAJECTORIES': no_of_trajectories, 
        'CONST_VEL_MODEL_PROB': 0.5, 
        'STOP_PROB': 0.05, 
        'DISCOUNT_AVG_PROB': 1.0, 
        'DISCOUNT_LOWER_BOUND': 0.2, 
        'VELOCITY_CHANGE_PROB': 0.1,
        'VELOCITY_CHANGE_NOISE': 0.05, 
        'ANGLE_CHANGE_PROB': 0.25, 
        'ANGLE_CHANGE_NOISE': 2, 
        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
    }

    univ_params = {
        'NOISE': 0.05, 
        'NO_OF_TRAJECTORIES': no_of_trajectories, 
        'CONST_VEL_MODEL_PROB': 0.5, 
        'STOP_PROB': 0.05, 
        'DISCOUNT_AVG_PROB': 1.0, 
        'DISCOUNT_LOWER_BOUND': 0.2, 
        'VELOCITY_CHANGE_PROB': 0.2,
        'VELOCITY_CHANGE_NOISE': 0.1, 
        'ANGLE_CHANGE_PROB': 0.2, 
        'ANGLE_CHANGE_NOISE': 2.5, 
        'GROUP_PERCENTAGES': [0.1, 0.4, 0.65, 0.85, 1.0], 
        'GROUP_CLUSTER_COUNT': [1, 6, 5, 5, 3]
    }


    our_method_params = [eth_params, hotel_params, zara1_params, zara2_params, univ_params]
    
    start = time.time()
    res = evaluate_all_datasets(our_method_params, files, trajectron_resultset_names, trajectron_ar3_resultset_names, evaluate_most_likely=True)
    end = time.time()
    elapsed_time = end-start
    print("Number of trajectories: ", no_of_trajectories)
    print("Elapsed time over all datasets: ", elapsed_time)
    print()
    all_results.append(res)



CVM - univ 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [00:40<00:00, 13.40it/s]


Number of trajectories:  100
Elapsed time over all datasets:  1879.6492722034454



CVM - univ 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [00:40<00:00, 13.40it/s]


Number of trajectories:  300
Elapsed time over all datasets:  2861.5835812091827



CVM - univ 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [00:40<00:00, 13.50it/s]


Number of trajectories:  1000
Elapsed time over all datasets:  6519.725186109543



CVM - univ 2: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 541/541 [00:40<00:00, 13.47it/s]


Number of trajectories:  2000
Elapsed time over all datasets:  12556.354624032974



Ours - univ 1:  81%|██████████████████████████████████████████████████████████████████████████▏                 | 358/444 [3:23:58<48:59, 34.18s/it]


KeyboardInterrupt: 

In [146]:
no_of_traj = [100, 300, 1000, 2000]
for idx, res in enumerate(all_results):
    print("No of generated trajectories: ", no_of_traj[idx])
    
    ours_results = res[0]
    trajectron_results = res[1]
    trajectron_ar3_results = res[2]
    cvm_long_results = res[3]
    cvm_short_results = res[4]

    index = [
    'ETH', 
    'Hotel', 
    'Zara 1', 
    'Zara 2',
    'Univ'
    ]

    df_data_best_of_20_fde = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['FDE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['FDE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['FDE'], index = index),
        #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['FDE'], index = index),
        'Ours': pd.Series(ours_results['BEST_OF_20']['FDE'], index = index)
    }

    df_best_of_20_fde = pd.DataFrame(df_data_best_of_20_fde)
    print(df_best_of_20_fde.to_markdown()) 

    df_data_best_of_20_ade = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['BEST_OF_20']['ADE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['BEST_OF_20']['ADE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['BEST_OF_20']['ADE'], index = index),
        #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['BEST_OF_20']['ADE'], index = index),
        'Ours': pd.Series(ours_results['BEST_OF_20']['ADE'], index = index)
    }

    df_best_of_20_ade = pd.DataFrame(df_data_best_of_20_ade)

    df_data_most_likely_fde = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['FDE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['FDE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['FDE'], index = index),
        #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['FDE'], index = index),
        'Ours': pd.Series(ours_results['MOST_LIKELY']['FDE'], index = index)
    }

    df_most_likely_fde = pd.DataFrame(df_data_most_likely_fde)

    df_data_most_likely_ade = {
        'CVM (8pt history)' : pd.Series(cvm_long_results['MOST_LIKELY']['ADE'], index = index),
        'CVM (2pt history)' : pd.Series(cvm_short_results['MOST_LIKELY']['ADE'], index = index),
        'Trajectron++' : pd.Series(trajectron_results['MOST_LIKELY']['ADE'], index = index),
        #'Trajectron++ AR3' : pd.Series(trajectron_ar3_results['MOST_LIKELY']['ADE'], index = index),
        'Ours': pd.Series(ours_results['MOST_LIKELY']['ADE'], index = index)
    }

    df_most_likely_ade = pd.DataFrame(df_data_most_likely_ade)
    print()

No of generated trajectories:  100
|        |   CVM (8pt history) |   CVM (2pt history) |   Trajectron++ |     Ours |
|:-------|--------------------:|--------------------:|---------------:|---------:|
| ETH    |            1.0962   |            1.7377   |       0.812384 | 0.675871 |
| Hotel  |            0.37131  |            0.580372 |       0.207567 | 0.207551 |
| Zara 1 |            0.807013 |            0.881145 |       0.326369 | 0.472738 |
| Zara 2 |            0.560829 |            0.647838 |       0.255412 | 0.344145 |
| Univ   |            0.884348 |            1.0586   |       0.444045 | 0.46086  |

No of generated trajectories:  300
|        |   CVM (8pt history) |   CVM (2pt history) |   Trajectron++ |     Ours |
|:-------|--------------------:|--------------------:|---------------:|---------:|
| ETH    |            1.13441  |            1.78385  |       0.812384 | 0.621228 |
| Hotel  |            0.367412 |            0.581255 |       0.207567 | 0.191977 |
| Zara 1 |      